In [22]:
import numpy as np
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from matplotlib import pyplot
from keras.models import load_model
import os
from pydub import AudioSegment
import scipy.io.wavfile
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.playback import play
import re
from scipy.io.wavfile import write

In [23]:
# convert mp3 audio file
recordings = os.listdir("Audio_Recordings_Flipkart")

In [32]:
for r in recordings:
    
    output = os.system(f"""ffmpeg -i "Audio_Recordings_Flipkart/"{r}  "Converted_Recordings_Flipkart/"{r[:-4]}.wav""")
    print(str(output)+" "+r)

0 0.mp3
0 1.mp3
0 10.mp3
0 100.mp3
0 101.mp3
0 102.mp3
0 103.mp3
0 104.mp3
0 105.mp3
0 106.mp3
0 107.mp3
0 108.mp3
0 109.mp3
0 11.mp3
0 110.mp3
0 111.mp3
0 112.mp3
0 113.mp3
0 114.mp3
0 115.mp3
0 116.mp3
0 117.mp3
0 118.mp3
0 119.mp3
0 12.mp3
0 120.mp3
0 121.mp3
0 122.mp3
0 123.mp3
0 124.mp3
0 125.mp3
0 126.mp3
0 127.mp3
0 128.mp3
0 129.mp3
0 13.mp3
0 130.mp3
0 131.mp3
0 132.mp3
0 133.mp3
0 134.mp3
0 135.mp3
0 136.mp3
0 137.mp3
0 138.mp3
0 139.mp3
0 14.mp3
0 140.mp3
0 141.mp3
0 142.mp3
0 143.mp3
0 144.mp3
0 145.mp3
0 146.mp3
0 147.mp3
0 148.mp3
0 149.mp3
0 15.mp3
0 150.mp3
0 151.mp3
0 152.mp3
0 153.mp3
0 154.mp3
0 155.mp3
0 156.mp3
0 157.mp3
0 158.mp3
0 159.mp3
0 16.mp3
0 160.mp3
0 161.mp3
0 162.mp3
0 163.mp3
0 164.mp3
0 165.mp3
0 166.mp3
0 167.mp3
0 168.mp3
0 169.mp3
0 17.mp3
0 170.mp3
0 171.mp3
0 172.mp3
0 173.mp3
0 174.mp3
0 175.mp3
0 176.mp3
0 177.mp3
0 178.mp3
0 179.mp3
0 18.mp3
0 180.mp3
0 181.mp3
0 182.mp3
0 183.mp3
0 184.mp3
0 185.mp3
0 186.mp3
0 187.mp3
0 188.mp3
0 189.mp3
0 1

In [25]:
#define optimiser
import keras
opt = Adam(lr=0.0002, beta_1=0.5)
#load Model
g_model = keras.models.load_model('g_model_new3(2).h5')

In [26]:
#Compile
g_model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])

In [27]:
import librosa

In [ ]:
audiofile = "1.wav"

In [ ]:
audio,sr = librosa.load("Converted_Recordings_Flipkart/" + audiofile,16000)
n_fft = 511
hop_length_fft = 63
frame_length = 8064*2
chunk_mag = []
chunk_pha = []
i = 0
while (i+frame_length<len(audio)):
    stftaudio = librosa.stft(audio[i:i+frame_length], n_fft=n_fft, hop_length=hop_length_fft)
    stftaudio_magnitude, stftaudio_phase = librosa.magphase(stftaudio)

    stftaudio_magnitude_db = librosa.amplitude_to_db(
        stftaudio_magnitude, ref=np.max)
    chunk_mag.append(stftaudio_magnitude_db/80)
    chunk_pha.append(stftaudio_phase)
    i += frame_length


In [ ]:
test_x = np.array(chunk_mag).reshape([len(chunk_mag),256,256,1])

In [ ]:
pred_y = g_model.predict(test_x)

In [ ]:
np.shape(pred_y)

In [6]:
def magnitude_db_and_phase_to_audio(frame_length, hop_length_fft, stftaudio_magnitude_db, stftaudio_phase):

    stftaudio_magnitude_rev = librosa.db_to_amplitude(stftaudio_magnitude_db, ref=1.0)

    audio_reverse_stft = stftaudio_magnitude_rev * stftaudio_phase
    audio_reconstruct = librosa.core.istft(audio_reverse_stft, hop_length=hop_length_fft, length=frame_length)

    return audio_reconstruct

In [7]:
def matrix_spectrogram_to_numpy_audio(m_mag_db, m_phase, frame_length, hop_length_fft)  :

    list_audio = []

    nb_spec = m_mag_db.shape[0]

    for i in range(nb_spec):

        audio_reconstruct = magnitude_db_and_phase_to_audio(frame_length, hop_length_fft, m_mag_db[i], m_phase[i])
        list_audio.append(audio_reconstruct)

    return np.vstack(list_audio)

In [8]:
audio_denoise_recons = matrix_spectrogram_to_numpy_audio(pred_y[:,:,:,0]*80, np.array(chunk_pha), frame_length, hop_length_fft)
nb_samples = audio_denoise_recons.shape[0]
denoise_long = audio_denoise_recons.reshape(1, nb_samples * frame_length)*10

NameError: name 'pred_y' is not defined

In [ ]:
import matplotlib.pyplot as plt
plt.plot(denoise_long[0, :])

In [ ]:
audio,sr = librosa.load("Converted_Recordings_Flipkart/" + audiofile,16000)

In [ ]:
plt.plot(audio)

In [ ]:
import IPython.display as ipd
ipd.Audio(data = denoise_long[0, :], rate = sr )

In [ ]:
import IPython.display as ipd
ipd.Audio(data = audio, rate = sr )

### Batch conversion from here

In [ ]:
scipy.io.wavfile.write('Audio_Output_Flipkart/'+audiofile, sr, denoise_long[0, :])

In [33]:
converted_recordings = os.listdir("Converted_Recordings_Flipkart")

In [34]:
len(converted_recordings)

745

In [46]:
converted_recordings[523]

'57.wav'

In [48]:
for cr in converted_recordings:
    
    
    sound = AudioSegment.from_wav("Converted_Recordings_Flipkart/" + cr)
    dBFS = sound.dBFS
    chunks = split_on_silence(sound, 
        min_silence_len = 500,
        silence_thresh = dBFS-10)

    res = chunks[0]
    for i in range(1,len(chunks)):
        res += chunks[i]
    res +=  AudioSegment.silent(duration=500)
    res.export("Converted_Recordings_Flipkart/" + cr, format = 'wav')

    audio,sr = librosa.load("Converted_Recordings_Flipkart/" + cr,16000)
    n_fft = 511
    hop_length_fft = 63
    frame_length = 8064*2
    chunk_mag = []
    chunk_pha = []
    i = 0
    while (i+frame_length<len(audio)):
        stftaudio = librosa.stft(audio[i:i+frame_length], n_fft=n_fft, hop_length=hop_length_fft)
        stftaudio_magnitude, stftaudio_phase = librosa.magphase(stftaudio)

        stftaudio_magnitude_db = librosa.amplitude_to_db(
            stftaudio_magnitude, ref=np.max)
        chunk_mag.append(stftaudio_magnitude_db/80)
        chunk_pha.append(stftaudio_phase)
        i += frame_length
        
    test_x = np.array(chunk_mag).reshape([len(chunk_mag),256,256,1])
    pred_y = g_model.predict(test_x)
    audio_denoise_recons = matrix_spectrogram_to_numpy_audio(pred_y[:,:,:,0]*80, np.array(chunk_pha), frame_length, hop_length_fft)
    nb_samples = audio_denoise_recons.shape[0]
    denoise_long = audio_denoise_recons.reshape(1, nb_samples * frame_length)*10
    
    scipy.io.wavfile.write('Audio_Output_Flipkart_New/'+cr, sr, denoise_long[0, :])
    
    #Post-processing
    sound = AudioSegment.from_wav('Audio_Output_Flipkart_New/' + cr)
    dBFS = sound.dBFS
    chunks = split_on_silence(sound, 
    min_silence_len = 1000,
    silence_thresh = dBFS-5)
    
    max_val = 0
    idx = 0
    for i in range(len(chunks)):
        if(chunks[i].max>max_val):
            idx = i
            max_val = chunks[i].max
    chunks[idx].export('Audio_Output_Flipkart_New/'+cr, format = 'wav')

    print(cr)


0.wav
1.wav
10.wav
100.wav
101.wav
102.wav
103.wav
104.wav
105.wav
106.wav
107.wav
108.wav
109.wav
11.wav
110.wav
111.wav
112.wav
113.wav
114.wav
115.wav
116.wav
117.wav
118.wav
119.wav
12.wav
120.wav
121.wav
122.wav
123.wav
124.wav
125.wav
126.wav
127.wav
128.wav
129.wav
13.wav
130.wav
131.wav
132.wav
133.wav
134.wav
135.wav
136.wav
137.wav
138.wav
139.wav
14.wav
140.wav
141.wav
142.wav
143.wav
144.wav
145.wav
146.wav
147.wav
148.wav
149.wav
15.wav
150.wav
151.wav
152.wav
153.wav
154.wav
155.wav
156.wav
157.wav
158.wav
159.wav
16.wav
160.wav
161.wav
162.wav
163.wav
164.wav
165.wav
166.wav
167.wav
168.wav
169.wav
17.wav
170.wav
171.wav
172.wav
173.wav
174.wav
175.wav
176.wav
177.wav
178.wav
179.wav
18.wav
180.wav
181.wav
182.wav
183.wav
184.wav
185.wav
186.wav
187.wav
188.wav
189.wav
19.wav
190.wav
191.wav
192.wav
193.wav
194.wav
195.wav
196.wav
197.wav
198.wav
199.wav
2.wav
20.wav
200.wav
201.wav
202.wav
203.wav
204.wav
205.wav
206.wav
207.wav
208.wav
209.wav
21.wav
210.wav
211.wav
21